In [ ]:
IBM_Key = kSctzVu5LXmSAPddZCn1P3koV9ICYDH_HSRykIgvastW

In [13]:
!pip install langdetect

  Stored in directory: C:\Users\xingxuan_dell\AppData\Local\pip\Cache\wheels\8d\b3\aa\6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [14]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

# Raw Data Combination & Preprocessing

In [17]:
# Read raw datas from the raw data file
path = r'D:\Winter 2020\COVID2019\tweet0120_0220\\'
files = os.listdir(path)

In [19]:
covid_twitter_data = pd.DataFrame()
# Concat the Twitters data into one-table
for file in files:
    data = pd.read_csv(str(path) + file)
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [27]:
os.chdir(r'D:\Winter 2020\COVID2019')

In [21]:
# Save the dataset
covid_twitter_data.to_csv('covid2019_0120_0220.csv',index=False)

In [23]:
# Clean text 
t = time.process_time()
clean_text = []
non_en = []
error = []
index = 0
for item in covid_twitter_data['text']:
    try:
        index = index +1
        # Delete url in text
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        # Delete @
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try:
            # Detect whether twitter is in English or not 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [28]:
# Save error tweets 
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv('error_df.csv',index=False)
non_en_df.to_csv('non_en_df.csv',index=False)

In [29]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([covid_twitter_data, clean_text], axis=1)
data.to_csv('covid2019_0120_0220_cleans.csv',index=False)
data.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,clean_text
0,False,[],[],False,True,0,[],1.219387e+18,0,"[{'screen_name': 'JapaDoBonor0', 'user_id': '3...",...,@JapaDoBonor0 o que é esse coronavirus? E se ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:57,1579564797,1219409258372046848,/TBorghi/status/1219409258372046848,274278103,Thiago Borghi,NaN,None
1,True,"['coronavirus', 'popalongnowpic']",['https://pbs.twimg.com/media/EOw2PUZXkAANlgu....,False,False,0,[],NaN,0,[],...,I’ve cancelled my delivery! #coronavirus #pop...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:53,1579564793,1219409242354069504,/exexpatkaren/status/1219409242354069504,40506876,Karen Owen,NaN,I’ve cancelled my delivery! #coronavirus #pop...
2,False,['coronavirus'],[],False,False,0,[],NaN,0,[],...,"Lo último que necesitaba saber, un nuevo virus...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:48,1579564788,1219409220015214592,/JManuelAlamilla/status/1219409220015214592,836304792062406656,JesMa,NaN,None
3,False,[],[],False,False,0,['https://bit.ly/2NJ9DD2'],NaN,0,[],...,"China: el ""coronavirus"" se transmite de person...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:45,1579564785,1219409206954078209,/adbradomin/status/1219409206954078209,1732410342,angel,NaN,None
4,False,[],[],False,False,0,[],NaN,0,[],...,Coronavirus é aquele vírus da Corona cerveja m...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:42,1579564782,1219409194429960192,/ltsroque/status/1219409194429960192,331665924,The Roque™ 🐻,NaN,None


In [30]:
# Drop the row when clean_text is none
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))

In [31]:
# random select 2500 samples
random.seed(9001)

data_api, data_test = train_test_split(data_strip_none, train_size=14500, random_state=0)

# Reset index
data_api = data_api.set_index(pd.Index(range(0,len(data_api))))
data_test = data_test.set_index(pd.Index(range(0,len(data_test))))

# Save the dataset
data_api.to_csv('data_api.csv',index=False)
data_test.to_csv('data_test_0120_0220.csv',index=False)

# IBM Tone Analyzer

In [150]:
data_api = pd.read_csv('data_api.csv')

In [151]:
data_api_12000 = data_api.iloc[9500:12000,]

In [153]:
data_api_12000.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,clean_text
9500,False,[],[],False,False,0,['https://www.wsj.com/articles/china-marshals-...,NaN,0,[],...,China Marshals Its Surveillance Powers Against...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-04 22:32:46,1580855566,1224823136350113793,/cestleiden/status/1224823136350113793,1715434532,YU HAO YU,NaN,China Marshals Its Surveillance Powers Against...
9501,False,"['nCoV', 'coronavirus']",[],False,False,1,['https://twitter.com/CDPHDirector/status/1224...,NaN,0,[],...,See the latest update on #nCoV #coronavirus fr...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-04 22:49:06,1580856546,1224827247325929472,/CA_EMSA/status/1224827247325929472,51667182,EMSA,NaN,See the latest update on #nCoV #coronavirus fr...
9502,False,"['Gold', 'China', 'RiskAppetite', 'Commodities...",[],False,False,1,['https://s68mv.app.goo.gl/SwgKs'],NaN,0,[],...,"Gold: On the back foot below $1,570 as coronav...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-11 23:53:19,1581465199,1227380122082258945,/FXstreetNews/status/1227380122082258945,27652717,FXStreet News,NaN,"Gold: On the back foot below $1,570 as coronav..."
9503,False,[],[],False,False,0,['https://www.bitchute.com/video/ZFKykJUTYlM/'],NaN,0,[],...,"Coronavirus Update: 31500 Infected, Zheizang a...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-07 22:08:21,1581113301,1225904153748287494,/Southcb007/status/1225904153748287494,3751906517,Libertarian Promotions,NaN,"Coronavirus Update: 31500 Infected, Zheizang a..."
9504,False,[],[],False,True,0,['https://twitter.com/CoronaTurkey/status/1226...,1.226254e+18,0,"[{'screen_name': 'russian_market', 'user_id': ...",...,kosovo first case of coronavirus will be annou...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-08 21:20:40,1581196840,1226254544641941504,/izleyici_2/status/1226254544641941504,450136150,izleyici2,NaN,kosovo first case of coronavirus will be annou...


In [154]:
authenticator = IAMAuthenticator('-------------api_key---------------')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('---------------api_url--------------------')
tone_analyzer.set_disable_ssl_verification(False)

In [36]:
# tone_analyzer.set_detailed_response(True)
# response = tone_analyzer.methodName(parameters)
# # Access response from methodName
# print(json.dumps(response.get_result(), indent=2))
# # Access information in response headers
# print(response.get_headers())
# # Access HTTP response status
# print(response.get_status_code())

In [38]:
# try:
#     print(tone_analyzer.tone('Trump is a foolish', content_type='text/plain') )
# except ApiException as ex:
#     print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [155]:
tones = []
index = 0
for item in data_api_12000['clean_text']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [156]:
tones_copy = tones.copy()
tones_copy = pd.DataFrame(tones_copy)

In [157]:
tones_copy.head()

,document_tone,sentences_tone
0,{'tones': []},NaN
1,{'tones': []},NaN
2,"{'tones': [{'score': 0.882584, 'tone_id': 'fea...",NaN
3,{'tones': []},NaN
4,"{'tones': [{'score': 0.506763, 'tone_id': 'ana...",NaN


In [158]:
data_api_12000 = data_api_12000.reset_index()

In [159]:
data_api_12000 = data_api_12000.drop(['index'], axis=1)

In [160]:
data_api_12000 = pd.concat([data_api_12000,tones_copy],axis=1)
data_api_12000.to_csv('data_api_12000.csv',index=False)

# Formating the document tones

In [161]:
data = pd.read_csv('data_api_12000.csv')

In [162]:
data.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,clean_text,document_tone,sentences_tone
0,False,[],[],False,False,0,['https://www.wsj.com/articles/china-marshals-...,NaN,0,[],...,2020-02-04 22:32:46,1580855566,1224823136350113793,/cestleiden/status/1224823136350113793,1715434532,YU HAO YU,NaN,China Marshals Its Surveillance Powers Against...,{'tones': []},NaN
1,False,"['nCoV', 'coronavirus']",[],False,False,1,['https://twitter.com/CDPHDirector/status/1224...,NaN,0,[],...,2020-02-04 22:49:06,1580856546,1224827247325929472,/CA_EMSA/status/1224827247325929472,51667182,EMSA,NaN,See the latest update on #nCoV #coronavirus fr...,{'tones': []},NaN
2,False,"['Gold', 'China', 'RiskAppetite', 'Commodities...",[],False,False,1,['https://s68mv.app.goo.gl/SwgKs'],NaN,0,[],...,2020-02-11 23:53:19,1581465199,1227380122082258945,/FXstreetNews/status/1227380122082258945,27652717,FXStreet News,NaN,"Gold: On the back foot below $1,570 as coronav...","{'tones': [{'score': 0.882584, 'tone_id': 'fea...",NaN
3,False,[],[],False,False,0,['https://www.bitchute.com/video/ZFKykJUTYlM/'],NaN,0,[],...,2020-02-07 22:08:21,1581113301,1225904153748287494,/Southcb007/status/1225904153748287494,3751906517,Libertarian Promotions,NaN,"Coronavirus Update: 31500 Infected, Zheizang a...",{'tones': []},NaN
4,False,[],[],False,True,0,['https://twitter.com/CoronaTurkey/status/1226...,1.226254e+18,0,"[{'screen_name': 'russian_market', 'user_id': ...",...,2020-02-08 21:20:40,1581196840,1226254544641941504,/izleyici_2/status/1226254544641941504,450136150,izleyici2,NaN,kosovo first case of coronavirus will be annou...,"{'tones': [{'score': 0.506763, 'tone_id': 'ana...",NaN


In [131]:
data = data.iloc[:,1:26]

In [163]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [164]:
# split sentiments in corresponding columns
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1
data.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,clean_text,sentences_tone,document_tones,fear,analytical,tentative,confident,joy,sadness,anger
0,False,[],[],False,False,0,['https://www.wsj.com/articles/china-marshals-...,NaN,0,[],...,China Marshals Its Surveillance Powers Against...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,"['nCoV', 'coronavirus']",[],False,False,1,['https://twitter.com/CDPHDirector/status/1224...,NaN,0,[],...,See the latest update on #nCoV #coronavirus fr...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,"['Gold', 'China', 'RiskAppetite', 'Commodities...",[],False,False,1,['https://s68mv.app.goo.gl/SwgKs'],NaN,0,[],...,"Gold: On the back foot below $1,570 as coronav...",NaN,"[{'score': 0.882584, 'tone_id': 'fear', 'tone_...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,False,[],[],False,False,0,['https://www.bitchute.com/video/ZFKykJUTYlM/'],NaN,0,[],...,"Coronavirus Update: 31500 Infected, Zheizang a...",NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,[],[],False,True,0,['https://twitter.com/CoronaTurkey/status/1226...,1.226254e+18,0,"[{'screen_name': 'russian_market', 'user_id': ...",...,kosovo first case of coronavirus will be annou...,NaN,"[{'score': 0.506763, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [165]:
data.to_csv('sentiment_12000.csv',index=False)

# Combining all datasets with tones analyzed by IBM

In [166]:
path = r'D:\Winter 2020\COVID2019\tweet_sentiment\\'
files = os.listdir(path)

In [169]:
files

['sentiment_12000.csv',
 'sentiment_2000.csv',
 'sentiment_4500.csv',
 'sentiment_7000.csv',
 'sentiment_9500.csv']

In [177]:
pd.read_csv(str(path) + 'sentiment_7000.csv',encoding='cp1252')

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,clean_text,sentences_tone,document_tones,tentative,analytical,confident,sadness,anger,joy,fear
0,False,"['Covid19', 'coronavirus', 'CoronavirusOutbreak']",[],True,True,4,[],1.230000e+18,1,"[{'screen_name': 'RecordGuyBln', 'user_id': '1...",...,"The infection rate of #Covid19 is so high, 26%...","[{'sentence_id': 0, 'text': 'The infection rat...","[{'score': 0.820919, 'tone_id': 'tentative', '...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,False,[],[],False,False,0,['https://www.hawaiinewsnow.com/2020/02/14/gov...,NaN,0,[],...,State: Japanese visitor recently in Hawaii con...,NaN,"[{'score': 0.920855, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,False,[],[],False,False,0,['https://flip.it/Rt9yhL'],NaN,0,[],...,California lab says it discovered coronavirus ...,NaN,"[{'score': 0.882284, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,False,[],[],True,False,45,['https://twitter.com/JHKMMA/status/1216754396...,NaN,3,[],...,"Update! According to my sources, Contender Ser...","[{'sentence_id': 0, 'text': 'Update!', 'tones'...","[{'score': 0.952514, 'tone_id': 'analytical', ...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,False,[],[],True,False,2,[],NaN,1,[],...,i went to seattle for svt and there was someon...,NaN,"[{'score': 0.708604, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,1.0,NaN,NaN,NaN
5,False,[],[],False,False,1,[],NaN,0,[],...,Spokane Regional Health Officer Dr. Bob Lutz s...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,False,[],[],True,True,5,[],1.230000e+18,2,"[{'screen_name': 'LynAldenContact', 'user_id':...",...,Many countries exporting to China are not runn...,"[{'sentence_id': 0, 'text': 'Many countries ex...","[{'score': 0.50505, 'tone_id': 'sadness', 'ton...",NaN,1.0,NaN,1.0,NaN,NaN,NaN
7,False,[],[],False,False,0,['https://www.forbes.com/sites/lisettevoytko/2...,NaN,0,[],...,Trump Proposes 16% Cut To CDC As Global Number...,NaN,"[{'score': 0.571834, 'tone_id': 'sadness', 'to...",NaN,1.0,NaN,1.0,NaN,NaN,NaN
8,False,"['Lunar', 'NewYear', 'HongKong', 'Wuhan']",[],False,False,0,['https://www.nytimes.com/2020/01/25/world/asi...,NaN,0,[],...,#Lunar #NewYear celebrations are being cancele...,"[{'sentence_id': 0, 'text': '#Lunar #NewYear c...","[{'score': 0.640331, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
9,False,"['HONGKONG', 'WuhanCoronavius', 'CORONAVIRUS']",[],True,False,0,[],NaN,1,[],...,And still as supermarket opened THERE ARE NO T...,"[{'sentence_id': 0, 'text': 'And still as supe...","[{'score': 0.786583, 'tone_id': 'anger', 'tone...",NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [180]:
covid_twitter_data = pd.DataFrame()
# Concat the Twitters data into one-table
for file in files:
    try:
        data = pd.read_csv(str(path) + file)
    except:
        data = pd.read_csv(str(path) + file,encoding='cp1252')
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [187]:
covid_twitter_data['fear'].sum()

847.0

In [182]:
# Save the dataset
covid_twitter_data.to_csv('sentiment12000full.csv',index=False)